
# Task 2: NGS Data Analysis

## Objective
This notebook demonstrates the process of aligning samples to the human genome, identifying somatic mutations, and estimating background mutation levels. The tasks align with the following sub-objectives:

### Sub-Tasks:
1. **Align the samples to the human genome using tools like BWA or Bowtie2** .
2. **Identify somatic mutations present in the cancer sample but absent in the normal tissue**:

   - Benchmark using established tools (e.g., Mutect2, VarScan2).

   - Develop custom code for mutation detection.
3. **Calculate the median background mutation level** and the reads per million required for confident mutation calling .

---


In [4]:
def count_variants(vcf_file):
    with open(vcf_file, 'r') as file:
        lines = file.readlines()
    
    # Count non-header lines (variants)
    variant_count = sum(1 for line in lines if not line.startswith('#'))
    return variant_count

# Example usage
snp_count = count_variants("C:/Users/Pavankumar/Downloads/ncbi_dataset/ncbi_dataset/data/GCF_000001405.40/somatic_output.snp.vcf")
indel_count = count_variants("C:/Users/Pavankumar/Downloads/ncbi_dataset/ncbi_dataset/data/GCF_000001405.40/somatic_output.indel.vcf")

print(f"Detected SNPs: {snp_count}")
print(f"Detected Indels: {indel_count}")


Detected SNPs: 0
Detected Indels: 0



## Step 1: Count Variants in VCF Files

### Description:
This code counts the number of Single Nucleotide Polymorphisms (SNPs) and Insertions/Deletions (Indels) from VCF files. This provides an overview of the variant distribution in the samples.
This code reads VCF files and loads variant data into a pandas DataFrame for further analysis. Each row in the DataFrame corresponds to a variant.

### Input:
- `somatic_output.snp.vcf`: SNPs detected in the sample.
- `somatic_output.indel.vcf`: Indels detected in the sample.

### Output:
The number of SNPs and Indels detected in the VCF files.


In [7]:
import pandas as pd

def read_vcf_to_dataframe(vcf_file):
    variant_data = []

    with open(vcf_file) as f:
        for line in f:
            if line.startswith('#'):
                continue  
            else:
                
                fields = line.strip().split('\t')
                variant_data.append(fields) 

  
    df = pd.DataFrame(variant_data, columns=['CHROM', 'POS', 'ID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO', 'FORMAT', 'NORMAL', 'TUMOR'])

    return df


snp_df = read_vcf_to_dataframe("C:/Users/Pavankumar/Downloads/ncbi_dataset/ncbi_dataset/data/GCF_000001405.40/somatic_output.snp.vcf")
indel_df = read_vcf_to_dataframe("C:/Users/Pavankumar/Downloads/ncbi_dataset/ncbi_dataset/data/GCF_000001405.40/somatic_output.indel.vcf")

print(f"Total SNPs: {len(snp_df)}")
print(f"Total Indels: {len(indel_df)}")


Total SNPs: 0
Total Indels: 0



## Steps 2: Calculate Background Mutation Level

### Description:
This section calculates the median background mutation level and determines the reads per million required to confidently call a mutation. The analysis uses the following inputs:

### Inputs:
- Normal tissue VCF file (`normal_variants.vcf`).

- Total reads from sequencing output (e.g., 1 million).

### Outputs:
- Median background mutation level.

- Reads per million required to detect a mutation.

### Purpose:
The background mutation level accounts for sequencing errors or biases, ensuring reliable mutation calling.

The analysis calculates:
1. **Median Background Mutation Level**: Reflects the frequency of mutations in normal tissue due to sequencing errors or biases.
2. **Reads per Million**: Determines the number of reads required to confidently identify mutations.


In [2]:
import numpy as np

def extract_variants_from_vcf(vcf_file):
    normal_variants = []  

    with open(vcf_file) as f:
        for line in f:
            if line.startswith('#'):
                continue  # Skip header lines
            else:
                fields = line.strip().split('\t')
                if len(fields) < 10:
                    continue
                
                variant_data = {
                    "chrom": fields[0],
                    "pos": fields[1],
                    "ref": fields[3],
                    "alt": fields[4],
                    "qual": fields[5],
                    "normal_genotype": fields[9]
                }
                normal_variants.append(variant_data) 

    return normal_variants

normal_variants = extract_variants_from_vcf("C:/Users/Pavankumar/Downloads/ncbi_dataset/ncbi_dataset/data/GCF_000001405.40/normal_variants.vcf/normal_variants.vcf")

def calculate_background_mutation_level(variants):
    return np.median([1 for _ in variants]) if variants else 0

median_background = calculate_background_mutation_level(normal_variants)
print(f"Median Background Mutation Level: {median_background}")

total_reads = 1000000
num_mutations = len(normal_variants) 

reads_per_million = total_reads / num_mutations if num_mutations > 0 else float('inf')
print(f"Reads per million required: {reads_per_million}")


Median Background Mutation Level: 1.0
Reads per million required: 1000000.0
